In [31]:
import os
import numpy as np
import open3d as o3d
from sklearn.cluster import DBSCAN
from scipy.spatial import KDTree
import cv2

import sys
sys.path.append("/Users/apple/Documents/UniversityDocument/Thesis/automative_self_driving_car/Autonomous-Navigation-and-Collision-Avoidance-Sytem-for-car/Collision_Avoidance")
from constants import *

from helpers.utils import *

obj_kmh = []
class PointCloudProcessor:
    def __init__(self, output_dir=None, store_frames=True, visualize=True, eps=EPS, min_samples=MIN_SAMPLES):
        self.eps = eps
        self.min_samples = min_samples
        self.visualize = visualize
        self.output_dir = output_dir
        self.store_frames = store_frames
        
        self.previous_centroids = []
        self.previous_cluster_ids = []

        # Initialize tracking history and IDs
        self.cluster_history = []
        self.tracking_id = 0
        self.cluster_color_map = {}
        self.previous_centroids_map = {}
        self.previous_directions_map = {}
        self.vehicle_type_cluster = {}
        
        self.my_car_cluster = []
        self.my_car = None
        
        if self.visualize == True:
            self.vis = o3d.visualization.Visualizer()
            self.vis.create_window(visible=visualize)
        
        self.frame_count = 0
    
    def process_frame(self, current_frame):
        
        clustering = DBSCAN(eps=self.eps, min_samples=self.min_samples).fit(current_frame)
        labels = clustering.labels_
        
        current_clusters_points = []
        current_centroids = []
        current_bboxs = []
        
        for cluster_label in set(labels):
            if cluster_label == -1:
                continue  # Skip noise points
            
            cluster_points = current_frame[labels == cluster_label]
            
            if np.any(cluster_points[:, 2] >= 2):
                continue
            
            cluster_pcd = o3d.geometry.PointCloud()
            cluster_pcd.points = o3d.utility.Vector3dVector(cluster_points)
            bbox = cluster_pcd.get_axis_aligned_bounding_box()
        
            if is_my_car(bbox.get_extent(), bbox.get_center()):
                self.my_car_cluster.append(cluster_points)
                
                if len(self.my_car_cluster) == 2:
                    self.my_car = merge_clusters(self.my_car_cluster)
                    my_car_pcd = o3d.geometry.PointCloud()
                    my_car_pcd.points = o3d.utility.Vector3dVector(self.my_car)
                    bbox_car = my_car_pcd.get_axis_aligned_bounding_box()
                    self.my_car_cluster.clear()

                    current_clusters_points.append(self.my_car)
                    current_centroids.append(bbox_car.get_center())
                    current_bboxs.append(bbox_car)
                    
                    # car_direction = calculate_car_direction(bbox_car)

                    # # Update the camera to follow the car's movement direction
                    # self.update_camera_view(car_direction, bbox_car.get_center())
            
                    continue
                    
            current_clusters_points.append(cluster_points)
            current_centroids.append(bbox.get_center())
            current_bboxs.append(bbox)
            
        current_centroids = np.array(current_centroids)
        
        if self.frame_count == 0:
            cluster_ids = np.arange(len(current_centroids))
            self.tracking_id = len(current_centroids)
        else:
            cluster_ids, self.tracking_id = self.track_clusters(current_centroids, self.previous_centroids, self.previous_cluster_ids)
        
        if self.visualize == True:
            self.visualize_frame(current_frame, current_clusters_points, current_bboxs, cluster_ids)
        
        self.previous_centroids = current_centroids
        self.previous_cluster_ids = cluster_ids
        
        self.frame_count += 1
            

    def visualize_frame(self, current_frame, cluster_points, bboxs, cluster_ids):
        all_points = []
        all_colors = []
        bounding_boxes = []
        centroids_spheres = []
        arrows = []
        previous_centroids_spheres = []
        
        for i, cluster in enumerate(cluster_points):
            self.assign_color(cluster_ids[i])
            
            cluster_pcd = o3d.geometry.PointCloud()
            cluster_pcd.points = o3d.utility.Vector3dVector(cluster)
            
            cluster_pcd = o3d.geometry.PointCloud()
            cluster_pcd.points = o3d.utility.Vector3dVector(cluster)
            
            bbox = bboxs[i]
            bbox_center = bbox.get_center()
            
            vehicle_type = classify_vehicle(bbox)
            
            if (cluster_ids[i] in self.vehicle_type_cluster) and ((vehicle_type == None) or vehicle_type == self.vehicle_type_cluster[cluster_ids[i]]):
                    vehicle_type = self.vehicle_type_cluster[cluster_ids[i]]
            elif (cluster_ids[i] in self.vehicle_type_cluster) and (vehicle_type != None) and vehicle_type != self.vehicle_type_cluster[cluster_ids[i]]:
                    self.vehicle_type_cluster[cluster_ids[i]] = vehicle_type
                    
                    
            if vehicle_type:
                bbox.color = (0, 1, 0)
                centroid_color = VEHICLE_CENTROID_COLORS[vehicle_type]
                # print(vehicle_type)
                
                if cluster_ids[i] in self.previous_centroids_map:
                    prev_centroid = self.previous_centroids_map[cluster_ids[i]]
                    previous_direction = self.previous_directions_map.get(cluster_ids[i], None)
                    
                    arrow_result = self.create_arrow(prev_centroid, bbox_center, bbox, previous_direction)
                    # print(arrow_result)
                    if arrow_result is not None:
                        area_mesh = arrow_result
                        arrows.append(area_mesh)

                    new_direction = bbox_center - prev_centroid
                    self.previous_directions_map[cluster_ids[i]] = new_direction  
                    
                    prev_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.05)
                    prev_sphere.translate(prev_centroid)
                    prev_sphere.paint_uniform_color([0.5, 0.5, 0.5])  
                    
                    previous_centroids_spheres.append(prev_sphere)
                    
                self.previous_centroids_map[cluster_ids[i]] = bbox_center
                
                bounding_boxes.append(bbox)
                all_colors.append(np.tile(centroid_color, (len(cluster), 1))) 
            
            else:
                centroid_color = [0, 0, 1]  
                all_colors.append(np.tile(centroid_color, (len(cluster), 1)))
                
            sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)
            sphere.translate(bbox_center)
            sphere.paint_uniform_color(centroid_color)
            
            centroids_spheres.append(sphere)
            all_points.append(cluster)
        
        # pcd = o3d.geometry.PointCloud()
        # pcd.points = o3d.utility.Vector3dVector(current_frame)
        # self.vis.clear_geometries()
        # entities = [pcd] + arrows 
        # for en in entities:
        #     self.vis.add_geometry(en)
        # self.vis.add_geometry(pcd)
        # self.vis.add_geometry(bounding_boxes)
        # for i in range(len(arrows)):
            # self.vis.add_geometry(arrows[i])

        # view_ctl = self.vis.get_view_control()
        # view_ctl.set_zoom(0.1)
        # view_ctl.set_lookat([0.0, 0.0, 0.0])
        # view_ctl.set_up([-1.0, 0.0, 0.0])
        # self.vis.poll_events()
        # self.vis.update_renderer()
        
        # if len(all_points) > 0:
        #     all_points = np.vstack(all_points)
        #     all_colors = np.vstack(all_colors)
            
        #     pcd = o3d.geometry.PointCloud()
        #     pcd.points = o3d.utility.Vector3dVector(all_points)
        #     pcd.colors = o3d.utility.Vector3dVector(all_colors)
            
        #     self.vis.clear_geometries()
        #     entities_to_draw = [pcd] + arrows

        #     for entity in entities_to_draw:
        #         self.vis.add_geometry(entity)

        #     view_ctl = self.vis.get_view_control()
        #     view_ctl.set_zoom(ZOOM_LIDAR_BIRD_EYE_PARAM)
        #     view_ctl.set_lookat([0.0, 0.0, 0.0])

        #     view_ctl.set_up([-1.0, 0.0, 0.0])
            
        #     self.vis.poll_events()
        #     self.vis.update_renderer()
            
        #     if self.store_frames == True and self.output_dir != None:

        #         frame_image = np.array(self.vis.capture_screen_float_buffer(True)) * 255
        #         frame_image = frame_image.astype(np.uint8)
                
        #         if not os.path.exists(self.output_dir):
        #             os.makedirs(self.output_dir)

        #         frame_filename = os.path.join(self.output_dir, f"frame_{self.frame_count:04d}.png")
        #         cv2.imwrite(frame_filename, frame_image)
        
    
    def close_vis(self):
        self.vis.destroy_window()
    
    def track_clusters(self, current_centroids, previous_centroids, prev_cluster_ids, threshold=DIST_THRESHOLD_TRACKING):
        if len(previous_centroids) == 0:
            return np.arange(len(current_centroids)) + self.tracking_id, self.tracking_id + len(current_centroids)
        
        # Create a KDTree for fast nearest neighbor lookup
        tree = KDTree(previous_centroids)
        
        # Assign new tracking IDs based on nearest neighbors from the previous frame
        assigned_ids = np.full(len(current_centroids), -1)  # -1 for new clusters
        distances, indices = tree.query(current_centroids)
        
        # Assign tracking IDs based on distance threshold
        for i, (dist, idx) in enumerate(zip(distances, indices)):
            if dist < threshold:
                assigned_ids[i] = prev_cluster_ids[idx]  # Match with existing cluster ID
        
        # Assign new IDs to unmatched clusters
        new_clusters = (assigned_ids == -1)
        assigned_ids[new_clusters] = np.arange(self.tracking_id, self.tracking_id + np.sum(new_clusters))
        
        return assigned_ids, self.tracking_id + np.sum(new_clusters)
        
    
    def assign_color(self, cluster_id):
        """Assigns a color to a cluster if not already assigned."""
        if cluster_id not in self.cluster_color_map:
            color_index = len(self.cluster_color_map) % len(COLOR_NAMES)  # Cycle through the color list
            self.cluster_color_map[cluster_id] = (COLOR_NAMES[color_index], COLOR_VALUES[COLOR_NAMES[color_index]])
    
    def create_arrow(self, start, end, bbox, previous_direction=None, scale_factor=ARROW_SCALE_FACTOR):
        """
        Creates an arrow mesh representing movement direction with a scaled length based on the magnitude of the vector.
        Uses Exponential Moving Average (EMA) for smoothing the direction.
        """
        kmh = find_velocity_kmh(end, start)
        obj_kmh.append(kmh)
        if kmh > VELOCITY_THRESHOLD and kmh <= 80:
            # Calculate the direction vector
            new_direction = end - start
            new_direction[2] = 0  # Ignore Z-axis movement for 2D plane movement visualization
    
            # If there is a previous direction, update it using exponential moving average
            if previous_direction is not None:
                updated_direction = 0.7 * previous_direction + 0.3 * new_direction  # Apply EMA
            else:
                updated_direction = new_direction
    
            # Calculate the magnitude (length) of the direction vector
            length = np.linalg.norm(updated_direction)
    
            if length == 0:
                return None  # No arrow needed if there's no movement
    
            # Scale the arrow length by a given factor
            scaled_length = length
    
            # Normalize the updated direction vector
            updated_direction_normalized = updated_direction / length
    
            # Create an arrow mesh with length proportional to the scaled movement
            # arrow1 = o3d.geometry.TriangleMesh.create_arrow(cylinder_radius=0.1, cone_radius=0.2,
            #                                                cylinder_height=scaled_length * 0.3, cone_height=scaled_length * 0.2)
            # arrow2 = o3d.geometry.TriangleMesh.create_arrow(cylinder_radius=0.1, cone_radius=0.2,
            #                                                cylinder_height=scaled_length * 0.3, cone_height=scaled_length * 0.2)
    
            # Align the arrow with the updated direction vector
            z_unit_vector = np.array([0, 0, 1])  # Upward vector
            axis = np.cross(z_unit_vector, updated_direction_normalized)
            axis_length = np.linalg.norm(axis)
    
            # if axis_length != 0:
            #     axis /= axis_length  # Normalize the axis of rotation
            #     angle = np.arccos(np.clip(np.dot(z_unit_vector, updated_direction_normalized), -1.0, 1.0))
            #     R = o3d.geometry.get_rotation_matrix_from_axis_angle(axis * angle)
                # arrow1.rotate(R, center=(0, 0, 0))
                # arrow2.rotate(R, center=(0, 0, 0))

            front_top_center, back_top_center = find_top_edge_centers(bbox)
            # Translate the arrow to start from the previous centroid (start position)
            # arrow1.translate(front_top_center)
            # arrow2.translate(back_top_center)
            arrow_color = [1, 0, 1]  
            # arrow1.paint_uniform_color(arrow_color)  # Set arrow color to purple
            # arrow2.paint_uniform_color(arrow_color)  # Set arrow color to purple
    
                    # Create area between the two arrows as a quadrilateral mesh
            area_mesh = create_area_between_arrows(front_top_center, back_top_center, updated_direction_normalized, scaled_length, arrow_color)
    
            return area_mesh
        
        return None


In [32]:
import os
import numpy as np
import open3d as o3d
from sklearn.cluster import DBSCAN
from scipy.spatial import KDTree
import cv2

import sys
sys.path.append("/Users/apple/Documents/UniversityDocument/Thesis/automative_self_driving_car/Autonomous-Navigation-and-Collision-Avoidance-Sytem-for-car/Collision_Avoidance")

from PointCloudProcessor import PointCloudProcessor

import constants

from constants import *
from helpers.utils import *


vis = o3d.visualization.Visualizer()
vis.create_window(visible=True)

def main():
    cloud_point_data = np.load('/Users/apple/Documents/UniversityDocument/Thesis/automative_self_driving_car/Autonomous-Navigation-and-Collision-Avoidance-Sytem-for-car/Collision_Avoidance/data/cloud_point_data2.pkl', allow_pickle=True)[1150:]
    output_dir = '/Users/apple/Documents/UniversityDocument/Thesis/automative_self_driving_car/Autonomous-Navigation-and-Collision-Avoidance-Sytem-for-car/Collision_Avoidance/frames'
    output_video = '/Users/apple/Documents/UniversityDocument/Thesis/automative_self_driving_car/Autonomous-Navigation-and-Collision-Avoidance-Sytem-for-car/Collision_Avoidance/output_videos/move_square_v4.mov'
    
    # for frame_idx in range(len(cloud_point_data)):
        # pcd = o3d.geometry.PointCloud()
        # pcd.points = o3d.utility.Vector3dVector(cloud_point_data[frame_idx])
        # vis.clear_geometries()
        # vis.add_geometry(pcd)
        # view_ctl = vis.get_view_control()
        # view_ctl.set_zoom(0.2)
        # view_ctl.set_lookat([0.0, 0.0, 0.0])
        # view_ctl.set_up([-1.0, 0.0, 0.0])
        # vis.poll_events()
        # vis.update_renderer()
    
    processor = PointCloudProcessor(output_dir)
        
    for frame_idx in range(len(cloud_point_data)):
        processor.process_frame(cloud_point_data[frame_idx])
    
    processor.close_vis()
        
main()

KeyboardInterrupt: 

: 

In [30]:
print(sorted(obj_kmh, reverse=True)[:50])

[769, 602, 498, 496, 469, 400, 374, 362, 362, 339, 337, 307, 283, 236, 236, 231, 195, 181, 180, 180, 177, 174, 171, 152, 152, 151, 146, 143, 143, 141, 138, 136, 136, 133, 133, 133, 131, 130, 130, 129, 129, 129, 128, 128, 127, 127, 126, 126, 126, 125]
